In [1]:
from argparse import ArgumentParser
import os
import shutil
from tqdm import tqdm
import numpy as np
import torch
from feature_extractors import AlexNetFC6, AlexNetConv5, AlexNetFull
from utils import listdir, image_to_tensor
from scipy.stats import pearsonr
from utils import p2r, stack_features, word2sense, regression, cv_regression_w2s, image_to_tensor, Subject, listdir, correlation

In [2]:
#feat_extractor = AlexNetFC6()
#feat_extractor = AlexNetConv5() # [stim, 256, 7, 7]
feat_extractor = AlexNetFull()

conditions = listdir('images')

# Setup w2s
w2s_file = "ThingsWrd2Sns.txt"
_, w2s_embeddings = word2sense(w2s_file)
nDim = 2250

condition_features = {}
for c in tqdm(conditions):
    c_name = c.split('/')[-1]
    stimuli = listdir(c)
    stimuli = [image_to_tensor(s, resolution=255) for s in stimuli]
    stimuli = torch.stack(stimuli) # stacked in 10
    #if torch.cuda.is_available():
    #    stimuli = stimuli.cuda()
    feats = []
    with torch.no_grad():
        _, c2, c3, c4, c5, fc6, _ = feat_extractor(stimuli)
    c2 = c2.mean(dim=0)
    c3 = c3.mean(dim=0)
    c4 = c4.mean(dim=0)
    c5 = c5.mean(dim=0)
    fc6 = fc6.mean(dim=0)
    feats.append(c2)
    feats.append(c3)
    feats.append(c4)
    feats.append(c5)
    feats.append(fc6)
    condition_features[c_name] = feats


100%|██████████| 1854/1854 [13:25<00:00,  2.30it/s]


In [3]:
np.amax()

# features = np.zeros((1470, 256)) #4096, 256
# voxels = np.zeros((1470, 200))
# w2s = np.zeros((1470, 2250))

# condition_features_ = torch.load('features_conv5.pth')

# #condition_features_ = torch.load('features_fc6.pth')
# print(len(condition_features_))

# for i, c in enumerate(conditions):
#     c_name = c.split('/')[-1]
#     if c_name in w2s_embeddings:
#         feature = condition_features_[c_name]
#         print(feature.shape)
#         voxel = np.load('./predicted_EVC_fc6' + '/' + c_name + '/' + c_name + '.npy')
#         #print(voxel.shape)
#         features[i, :] = feature
#         voxels[i, :] = voxel
#         w2s[i, :] = w2s_embeddings[c_name]


In [ ]:
corr_featvox = np.zeros((4096, 200))

for i in range(4096):
    for j in range(200):
        corr_featvox[i, j] = (correlation(features[:, i], voxels[:, j]))

In [ ]:
r_threshold = p2r(0.05/4096, 1470)

corr_featvox_t = corr_featvox > r_threshold

feat_t = np.zeros((4096, 1))

for i in range(4096):
    feat_t[i] = np.sum(corr_featvox_t[i, :])

# import matplotlib.pyplot as plt
# plt.figure(figsize=(50,50))
# plt.imshow(corr_featvox)
# plt.show()



In [ ]:
feat_argmax = ((feat_t.T).argsort())[0][::-1]

In [ ]:
feat_argmax_100 = feat_argmax[0:100]

In [ ]:
feat_random_100 = np.random.choice(4096, 100, replace=False)

feat_top = features[:, feat_argmax_100]
feat_random = features[:, feat_random_100]

feat_top_r = []
feat_random_r = []

In [ ]:
print ("cross-validation with top or random features")

_, voxel_mean_r = cv_regression_w2s(feat_top, w2s_embeddings, fit=None, k=9, l2=0.0, pc=None)

feat_top_r.append(voxel_mean_r)

_, voxel_mean_r = cv_regression_w2s(feat_random, w2s_embeddings, fit=None, k=9, l2=0.0, pc=None)

feat_random_r.append(voxel_mean_r)


In [ ]:
r_threshold = p2r(0.05/2250, 1470)

top_dim_set = {}
top_max_set = {}
top_mean_set = {}
top_med_set = {}

top_dim = []
top_max = []
top_mean = []
top_med = []
for dim_pred in feat_top_r:
    dim_pred = np.array(dim_pred)
    dim_pred_score = np.sum(dim_pred > r_threshold)
    dim_max = np.nanmax(dim_pred)
    dim_mean = np.nanmean(np.array(dim_pred))
    dim_med = np.nanmedian(np.array(dim_pred))
    top_dim.append(dim_pred_score)
    top_max.append(dim_max)
    top_mean.append(dim_mean)
    top_med.append(dim_med)
top_dim_set = top_dim
top_max_set = top_max
top_mean_set = top_mean
top_med_set = top_med

random_dim_set = {}
random_max_set = {}
random_mean_set = {}
random_med_set = {}


random_dim = []
random_max = []
random_mean = []
random_med = []
for dim_pred in feat_random_r:
    dim_pred = np.array(dim_pred)
    dim_pred_score = np.sum(dim_pred > r_threshold)
    dim_max = np.nanmax(dim_pred)
    dim_mean = np.nanmean(np.array(dim_pred))
    dim_med = np.nanmedian(np.array(dim_pred))
    random_dim.append(dim_pred_score)
    random_max.append(dim_max)
    random_mean.append(dim_mean)
    random_med.append(dim_med)
random_dim_set = random_dim 
random_max_set = random_max
random_mean_set = random_mean
random_med_set = random_med

In [ ]:
print(top_dim_set)
print(top_max_set)
print(top_mean_set)
print(top_med_set)
print(random_dim_set)
print(random_max_set)
print(random_mean_set)
print(random_med_set)

In [ ]:
pred_feat_top = np.zeros((100, 2250))
pred_feat_random = np.zeros((100, 2250))
r_threshold2 = p2r(0.05/2250, 1470)


for i in range(100):
    for j in range(2250):
        pred_feat_top[i, j] = (correlation(feat_top[:, i], w2s[:, j]))
        #pred_feat_random[i, j] = (correlation(feat_random[:, i], w2s[:, j]))

print("predicting w2s with top 100 fc6 features")

In [ ]:
pred_feat_top_score = np.zeros((100, 1))
for i in range(100):
    pred_feat_top_score[i] = (np.sum(pred_feat_top[i, :] > r_threshold2))

print((pred_feat_top_score.T)[0])



In [ ]:
pred_feat_random_score = np.zeros((100, 10))

for n in range(10):
    feat_randomarg = random.sample(range(4096), 100)
    feat_random = features[:, feat_randomarg]
    print(feat_random.shape)
    for i in range(100):
        for j in range(2250):
            #pred_feat_top[i, j] = (correlation(feat_top[:, i], w2s[:, j]))
            pred_feat_random[i, j] = (correlation(feat_random[:, i], w2s[:, j]))
    print("predicting w2s with random 100 fc6 features, " + str(n))
    #print(np.sum(pred_feat_random > r_threshold2)) 

In [ ]:

for n in range(10):
    for i in range(100):
        pred_feat_random_score[i, n] = (np.sum(pred_feat_random[i, :] > r_threshold2))
        #print(np.sum(pred_feat_random[i, :] > r_threshold2))


In [ ]:
print(np.sort(pred_feat_top_score.T))
print(pred_feat_random_score[:, 0])


fig = plt.figure(figsize=(20, 25))

#ax.bar(idx,r)
plt.bar(range(0,100), np.sort(pred_feat_top_score.T)[0][::-1], color='darkred', label='top', alpha=0.8)
for i in range(10):
    plt.bar(range(0,100), np.sort(pred_feat_random_score[:, i])[::-1], color='grey', label='random', alpha=0.1)

plt.legend(prop={'size': 20})

plt.show()